In [ ]:
!pip uninstall ultralytics -y

In [ ]:
!pip uninstall ultralytics-thop -y

In [ ]:
!pip uninstall numpy -y

In [ ]:
!pip install numpy==1.26.4

In [ ]:
!pip install ultralytics

In [ ]:
!pip uninstall torch torchvision torchaudio -y
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu118

In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
!pip install scikit-learn

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
from ultralytics import YOLO

In [ ]:
dataset_dir = r"C:\ai_project01\mask_yolo_dataset"
if os.path.exists(dataset_dir):
    shutil.rmtree(dataset_dir)
    print(f"기존 폴더 삭제: {dataset_dir}")
image_train_dir = os.path.join(dataset_dir, "images/train")
image_val_dir = os.path.join(dataset_dir, "images/val")
label_train_dir = os.path.join(dataset_dir, "labels/train")
label_val_dir = os.path.join(dataset_dir, "labels/val")
for d in [image_train_dir, image_val_dir, label_train_dir, label_val_dir]:
    os.makedirs(d, exist_ok=True)
image_src_dir = r"C:\ai_project01\mask_images"
label_src_dir = r"C:\ai_project01\labels"
image_files=[]
for subdir in ['mask_on', 'no_mask']:
    subdir_path = os.path.join(image_src_dir, subdir)
    for f in os.listdir(subdir_path):
        if f.lower().endswith(('jpg', 'png', 'jpeg')):
            full_file_path=os.path.join(subdir_path, f)
            image_files.append(full_file_path)
train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)
def move_files(files, image_dest, label_dest):
    for img_path in files:
        filename = os.path.basename(img_path)
        label_name = os.path.splitext(filename)[0] + '.txt'
        label_path = os.path.join(label_src_dir, label_name)
        if os.path.exists(label_path):
            shutil.copy2(img_path, image_dest)
            shutil.copy2(label_path, label_dest)
        else:
            print(f" !! 라벨 파일을 찾을 수 없습니다 : {img_path}")
move_files(train_files, image_train_dir, label_train_dir)
move_files(val_files, image_val_dir, label_val_dir)
print("데이터 분할 및 파일 이동이 모두 완료되었습니다!")

In [ ]:
data_yaml_path = os.path.join(dataset_dir, "data.yaml")
yaml_content = f"""
train : C:/ai_project01/mask_yolo_dataset/images/train
val : C:/ai_project01/mask_yolo_dataset/images/val

names:
  0: mask_on
  1: no_mask

nc: 2
"""

with open(data_yaml_path, 'w') as file:
    file.write(yaml_content.strip())

print(f" data.yaml 생성 완료 : {data_yaml_path}")


In [ ]:
model = YOLO('yolov8m.pt')
results = model.train(
    data=data_yaml_path,
    epochs=1000,
    imgsz=640,
    batch=16,
    name='mask_detection',
    project='C:/ai_project01/results',
    exist_ok=True,
    patience=30
)
print("\n 학습 완료!")